# Formula Student EV

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
from IPython.display import display
%matplotlib inline

### Nomenclature
- TS: Tractive system - Every part electrically connected to the motor and battery
- HVD: High Voltage disconnect - More comonly known as Master Safety Disconnect (MSD).
- TSAC: Tractive System Accumulator Container - More commonly known as Battery Box.
- TS Accumulator Segments - More commonly known as battery modules.
- AIR: Accumulator Isolation Relay - More commonly known as Contactor
- AMS: Accumulator Managment System - More commonly known as BMS
- IMD: Isolation Monitoring Device


## Important takes from 2021 FS Rules
- Battery can output a maximum of 80kW (107HP). (HV2.1.1)
- Regen is allowed (EV2.2.2)
- Overcurrent Protection is required. (Main Fuse) (EV3.2.7)
- A maximum 600V DC is allowed. And 630V DC when considering signal. (EV4.1.1)
- All components in the traction system must be rated for the maximum voltage. (EV4.1.2)
- Fans with >50W power must not be connected to the TS. ((EV4.1.4)
- HVIL is allowed. But HV and LV connections are not allowed on the same connector. (EV4.3.3)
- All HV components must be inside the roll structure, exception are hub motors. (EV4.4.1)
- All HV components must at >=3500mm from the ground must be protected from front, side or rear impacts. (EV4.4.2) 
- HV and LV are allowed on the same PCB, isolation devices (opto-isolators) may be used and established component spacing is required. (EV4.3.6)
- Electrical tape and rubber like coating is not allowed on TS components. (EV4.5.2)
- TS components must be waterproof (EV4.5.4)
- External HV cabling must have a grounded shield. (EV4.5.9)
- Plastic is not allowed in HV bolted connections except for FR-4. (EV4.5.12)
- External HV crimps must withstand 200N. (EV4.5.8)
- All electrical connections must have positive locking mechanisms. (EV4.5.13)
- **Soldered connections are not allowed, except for SMD devices. (EV4.5.15)**
- An event provided data logger must be used to log voltage and current. (EV4.6.1)
- 2 Tractive System Measuring Points must be placed next to the master switches. (EV4.7.1)
- An HVD is needed. (EV4.8.1)
- Battery to Inverter connections must be external and removable without tools and above 350mm from the ground. 
- Inverter must have an active discharge circuit that will drain voltage in the system when a disconnect is detected or power is turned off. (EV4.9)
- Battery modules must not exceed 120V, not be over 1.6kWh and weigh less than 12kg. (EV5.3.2)
- Interchangeable batteries are allowed at the competitions. (EV5.3.3)
- The battery box must be openable for technical inspection. (EV5.3.6)
- **The batteries must be accesible with a fire extinguisher (EV5.3.10)**
- Every Battery Box requires a Fuse and two contactors. (EV5.4.2)
- DCDC Converters are allowed inside the battery box. (EV5.4.3)
- **Battery modules must be disconnected without tools? (EV5.4.4)**
- Battery modules must be separated by isolating/fire retardant material. (EV5.4.6)
- Every wire in the battery box must be rated to the maximum voltage. (EV5.4.7) 
- The battery box cannot be part of the impact structure. (EV5.5.2)
- The battery box may be made out of any material, materials other than steel or aluminium they must be submitted in the SES.  (EV5.5.4)
- The battery pack requires a pressure release valve. (EV5.5.17)
- Pouch cell tabs must not carry any mechanical load. (EV5.5.10)
- A removable temperature logging device may be provided at the competition. (EV5.8.5) 
- Sensing Pre-charge Circuit is required. (EV5.7.1)
- The BMS must measure: (EV5.8.2)
    - Cell voltages
    - Current
    - Temperature of warmest cells 
    - Temperature of at least 30% of the cells, equally distributed. 
- Cell temperature must be measured on the negative terminal of the cell.(EV5.8.3) 
- An insulation fault detection device must be used. (EV6.3.2)
- Chargers must be presented at the technical inspection. (EV7.1.1)
- Team may build their own chargers. (EV7.1.3)
- Software Calibration and Charging are allowed after technical inspection. (IN1.5.1)
- The battery must be able to be disassembled at the technical inspection. (IN3.1.2)
- The car is submitted to a rain test. Comparable IP rating would be IP65. (IN9)


## Battery Specs
- For rear only motors Regen is around 30% of consumed energy. Close to 40% for 4wd. 
- Most teams calculate from previous telemetry that consumption is 9kWh during endurance. 
- Accounting for Regen most teams use battery packs between 6kWh and 6.5kWh. 
- ~30% of teams use pouch based cells, this percentage is higher for European Teams. Most teams use cylindrical 18650 cells. 
- Inside cylindrical cells most teams use Sony VTC6 for availability and higher energy density. 
- I personally have used Samsung 30T 21700 cells and they have very low internal resistance. 


### Cell Data

In [2]:
# Cell data indexed as 
dataind = {"CN":"Cell Model","RC":"Rated Capacity (mAh)","MV":"Max Voltage (V)","NV":"Nominal Voltage (V)",
           "IR":"Internal Resistance (mOhm)","CD":"Continous Discharge (A)", "W":"Weight (g)"}

cells = [{"CN":"Sony VTC6 18650", "RC":3000, "MV":4.2, "NV":3.6, "IR":21.4,"W":46.9,"CD":15},
          {"CN":"Samsung 30T 21700", "RC":3000, "MV":4.2, "NV":3.6, "IR":11.6,"W":68.1,"CD":35}]

cellFrame = pd.DataFrame.from_records(cells).transpose().rename(index = dataind)
cellFrame = cellFrame.rename(columns= cellFrame.iloc[0]).iloc[1:]
cellFrame


,Sony VTC6 18650,Samsung 30T 21700
Rated Capacity (mAh),3000,3000
Max Voltage (V),4.2,4.2
Nominal Voltage (V),3.6,3.6
Internal Resistance (mOhm),21.4,11.6
Weight (g),46.9,68.1
Continous Discharge (A),15,35


### Calculated Cell Data

In [3]:
@widgets.interact(NS=widgets.IntSlider(min=100, max=150, step=1, value=142, description="Series"),NP=widgets.IntSlider(min=1, max=6, step=1, value=4, description="Parallels"))
def f(NS,NP):
    tempval = pd.DataFrame(index=list(cellFrame),columns=['Total Number of cells','Nominal Capacity (kWh)','Max Pack Voltage(V)','Nominal Pack Voltage(V)',
                                                          'Series Current for 80kW @ Vnom (A)','Max Continous Power @ Vnom (kW)','Weight of the pack (kg)'])
    for i in range(len(cellFrame.columns)):
        NPV = cellFrame.loc[dataind['NV']][i]*NS # Nominal Pack Voltage (V)
        MPV = cellFrame.loc[dataind['MV']][i]*NS # Max Pack Voltage (V)
        NPC = cellFrame.loc[dataind['RC']][i]*NP*NPV/10**6 # Nominal Pack Capacity (kWh)
        C80kW = 80000/NPV/NP # Series Current for 80kW @ Nominal Voltage (A)
        MCP = cellFrame.loc[dataind['CD']][i]*NPV*NP/1000 # Max Continous Power @ Vnom (kW)
        PW = cellFrame.loc[dataind['W']][i]*NS*NP/1000 # Pack Weight (kg)
        tempval.iloc[i] = [NS*NP,NPC,MPV,NPV,C80kW,MCP,PW]
        
    return tempval.transpose()

interactive(children=(IntSlider(value=142, description='Series', max=150, min=100), IntSlider(value=4, descrip…

Visibly the Sony VTC6 is much lighter but at what cost? The low continous discharge rate is an important limiting factor on this battery, such that to generate 80kW at nominal voltage on a 142s-4p battery the cell would experience 39A or 2.6 times its rated discharge. The values above do not consider internal resistance, which would have a much larger effect on the Sony VTC6 vs the Samsung 30T 21700, in other words on hard pulls, i.e. acceleration event, the Sony would sag much more than the Samsung leading to a much lower peak performance.

Charging is also a factor to consider as the Samsung will be able to charge more than twice as fast to almost the same capacity. 

## References
- https://www.imeche.org/events/formula-student/team-information/rules
- https://www.facebook.com/batterymooch/posts/bench-retest-results-samsung-30t-21700an-incredible-35a-3000mah-batterybottom-li/2091705384452370/
- https://www.facebook.com/batterymooch/posts/2018639535092289
